Given a string s which represents an expression, evaluate this expression and return its value.

The integer division should truncate toward zero.

You may assume that the given expression is always valid. All intermediate results will be in the range of [-231, 231 - 1].

Note: You are not allowed to use any built-in function which evaluates strings as mathematical expressions, such as eval().



Example 1:

Input: s = "3+2*2"
Output: 7
Example 2:

Input: s = " 3/2 "
Output: 1
Example 3:

Input: s = " 3+5 / 2 "
Output: 5


Constraints:

1 <= s.length <= 3 * 10^5
s consists of integers and operators ('+', '-', '*', '/') separated by some number of spaces.
s represents a valid expression.
All the integers in the expression are non-negative integers in the range [0, 2^31 - 1].
The answer is guaranteed to fit in a 32-bit integer.

In [ ]:
from typing import *
from functools import lru_cache

In [1]:
class TreeNode:
    def __init__(self, val: Union[int, str], left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    @property
    def is_num(self)->bool:
        return isinstance(self.val, int)

class Solution:
    def calculate(self, s: str) -> int:

        # parse the string to be a list of nodes
        node_list = self.parse_string(s)

        # build the arithmetic tree
        root = self.build_tree(node_list)

        # evaluate the arithmetic tree
        return self.evaluate_tree(root)

    def parse_string(self, s: str)-> List[TreeNode]:

        stack = []
        node_list = []

        def _process_stack():
            total_sum = 0
            power = 0
            while len(stack) > 0:
                total_sum += stack.pop() * 10**power
                power += 1

            node_list.append(TreeNode(total_sum))

        for c in s:
            int_val = ord(c) - ord('0')
            if 0 <= int_val <= 9:
                stack.append(int_val)
            elif c == ' ':
                # ignore white-spaces
                continue
            else:
                # it must be that this is an operator in [+,-,*,/]
                _process_stack()
                node_list.append(TreeNode(c))

        _process_stack()

        return node_list

    def build_tree(self, node_list: List[TreeNode]) -> TreeNode:
        """
        Given a list of nodes where each node is either an integer or an operator,
        build an arithmetic tree that can be later evaluated. Currently, this tree
        only supports addition, subtraction, multiplication, and division. Parenthesis
        and exponentiation are not yet supported.
        :param node_list:   A list of nodes to form into a tree.
        :return:            The root node of the arithmetic tree.
        """

        # for simplicity, we take the first element (which should be a number),
        # and add it with 0
        root = TreeNode('+', left=TreeNode(0), right=node_list[0])

        for i in range(1, len(node_list), 2):
            # take two elements at a time in which the first element should be an operator,
            # and the second element should be a number
            op_node, num_node = node_list[i], node_list[i + 1]
            assert not op_node.is_num
            assert num_node.is_num

            op_node.right = num_node

            if op_node.val in  ['*', '/']:
                # We separate terms by addition and subtraction.
                # The current group of terms should have operators and numbers
                # append to their subtree if the operator involved multiplication/division.
                op_node.left = root.right
                root.right = op_node
            elif op_node.val in ['+', '-']:
                # We have encountered a new addition/subtraction operator. We are done fleshing
                # out our subtree at root.right, and we should update the root node
                op_node.left= root
                root = op_node
            else:
                raise NotImplementedError

        return root

    def evaluate_tree(self, node: TreeNode)->int:
        """
        We should evaluate our arithmetic tree and return the final result.
        Evaluating the tree correctly involves performing a post-traversal of the tree.
        :param node:    The root of the tree.
        :return:        An integer evaluation of the tree.
        """

        @lru_cache(maxsize=None)
        def _post_traversal(n: TreeNode) -> int:
            if n.is_num:
                return n.val

            left_val = _post_traversal(n.left)
            right_val = _post_traversal(n.right)

            if n.val == '+':
                return left_val + right_val
            elif n.val == '-':
                return left_val - right_val
            elif n.val == '*':
                return left_val * right_val
            elif n.val == '/':
                return int(left_val / right_val)
            else:
                raise NotImplementedError

        # perform a post-traversal on the tree
        return _post_traversal(node)

NameError: name 'Union' is not defined